In [ ]:
!pip install requests pandas beautifulsoup4 pycountry

In [ ]:
!wget -N https://download.geonames.org/export/dump/countryInfo.txt
!wget -N https://download.geonames.org/export/dump/admin1CodesASCII.txt
!wget -N https://download.geonames.org/export/dump/allCountries.zip
!unzip -o allCountries.zip

In [ ]:
import pandas as pd
import requests
import pycountry
from datetime import datetime
import os
import json

In [ ]:
CONTINENT_MAP = {
    'AS': 'Asia',
    'AF': 'Africa',
    'NA': 'North America',
    'SA': 'South America',
    'EU': 'Europe',
    'OC': 'Oceania',
    'AN': 'Antarctica'
}

BASE_PATH = '/content/geodata/'
os.makedirs(BASE_PATH, exist_ok=True)

In [ ]:
def get_continent_data():
  """
    Obtiene y procesa datos básicos de todos los continentes
    Args:
        None
    Returns:
        continents_data: DataFrame con información de los Continentes
  """
  print("Obteniendo datos de los continentes...")
  continents_data = []
  for idx, (code, name) in enumerate(CONTINENT_MAP.items(), start=1):
      continents_data.append({
          'id': idx,
          'code': code,
          'name': name,
          'is_active': 1,
          'created_at': datetime.now(),
          'updated_at': datetime.now()
      })
  return pd.DataFrame(continents_data)

In [ ]:
def get_countries_data():
  """
    Obtiene y procesa datos básicos de todos los países
    Args:
        None
    Returns:
        countries_data: DataFrame con información de los países
  """
  url = "https://restcountries.com/v3.1/all"
  try:
      print("Obteniendo datos de países...")
      response = requests.get(url, timeout=20)
      response.raise_for_status()
  except requests.exceptions.Timeout:
      print("La solicitud excedió el tiempo de espera.")
      return pd.DataFrame()
  except requests.exceptions.RequestException as e:
      print(f"Error al hacer la solicitud: {e}")
      return pd.DataFrame()

  data = response.json()

  country_data = []
  for country in data:
      try:
          region = country.get('region', '')
          continent_map = {
              'Asia': 'AS',
              'Africa': 'AF',
              'Americas': 'NA',
              'Europe': 'EU',
              'Oceania': 'OC',
              'Antarctic': 'AN'
          }
          continent_code = continent_map.get(region, 'OC')

          if region == 'Americas':
              subregion = country.get('subregion', '')
              if subregion and ('South' in subregion):
                  continent_code = 'SA'
              else:
                  continent_code = 'NA'

          if not country.get('cca2'):
              continue

          country_dict = {
              'alpha_2_code': country.get('cca2', ''),
              'alpha_3_code': country.get('cca3', ''),
              'numeric_code': country.get('ccn3', '000') if country.get('ccn3') else '000',
              'official_name': country.get('name', {}).get('official', ''),
              'common_name': country.get('name', {}).get('common', ''),
              'english_name': country.get('name', {}).get('official', ''),
              'continent_code': continent_code,
              'is_active': 1,
              'created_at': datetime.now(),
              'updated_at': datetime.now(),
              'capital': country.get('capital', []),
              'timezones': country.get('timezones', []),
              'callingCode': country.get('idd', {}).get('root', '') + (country.get('idd', {}).get('suffixes', [''])[0] if country.get('idd', {}).get('suffixes') else '')
          }

          if len(country_dict['alpha_2_code']) == 2:
              country_data.append(country_dict)

      except Exception as e:
          print(f"Error procesando país: {str(e)}")

  return pd.DataFrame(country_data)

In [ ]:
def get_countries_info_data(df_countries):
  """
    Obtiene y procesa datos básicos de todos los países
    Args:
        df_countries: DataFrame con la información de países
    Returns:
        countries_info_data: DataFrame con información de los países
  """
  print("Obteniendo información adicional de países...")
  country_info_data = []

  if 'continent_code' not in df_countries.columns:
      print("Advertencia: 'continent_code' no está en df_countries, agregando columna predeterminada")
      df_countries['continent_code'] = 'OC'

  for index, row in df_countries.iterrows():
      try:
          info_dict = {
              'country_alpha2': row['alpha_2_code'],
              'capital': ', '.join(row.get('capital', []) if isinstance(row.get('capital'), list) else []),
              'flag_png': f"https://flagcdn.com/w320/{row['alpha_2_code'].lower()}.png",
              'flag_svg': f"https://flagcdn.com/{row['alpha_2_code'].lower()}.svg",
              'calling_code': f"+{row.get('callingCode', '')}",
              'timezones': json.dumps(row.get('timezones', [])),
              'continent_code': row['continent_code'],
              'is_active': True,
              'created_at': datetime.now(),
              'updated_at': datetime.now()
          }
          country_info_data.append(info_dict)
      except Exception as e:
          print(f"Error en info país {row['alpha_2_code']}: {str(e)}")

  result_df = pd.DataFrame(country_info_data)
  if 'continent_code' not in result_df.columns:
      print("Error: 'continent_code' no está en el DataFrame resultante")
      result_df['continent_code'] = 'OC'

  return result_df

In [ ]:
def process_geonames_cities():
  """
    Obtiene y procesa datos básicos de todas las ciudades
    Args:
        None
    Returns:
        cities_data: DataFrame con información de las ciudades
  """
  print("Procesando ciudades...")
  columns = [
      'geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude',
      'feature_class', 'feature_code', 'country_code', 'cc2', 'admin1_code',
      'admin2_code', 'admin3_code', 'admin4_code', 'population', 'elevation',
      'dem', 'timezone', 'modification_date'
  ]

  chunks = []
  for chunk in pd.read_csv('allCountries.txt', sep='\t', names=columns, chunksize=100000, dtype=str):
      chunk = chunk[chunk['feature_class'] == 'P']
      chunks.append(chunk)

  df = pd.concat(chunks)

  df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
  df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
  df['population'] = pd.to_numeric(df['population'], errors='coerce').fillna(0)
  df['elevation'] = pd.to_numeric(df['elevation'], errors='coerce').fillna(0)

  df['is_capital'] = df['feature_code'].isin(['PPLC', 'PPLA'])
  df['is_state_capital'] = df['feature_code'].isin(['PPLA', 'PPLA2', 'PPLA3', 'PPLA4'])

  return df[[
      'geonameid', 'name', 'latitude', 'longitude',
      'country_code', 'admin1_code', 'population',
      'is_capital', 'is_state_capital', 'timezone'
  ]]

In [ ]:
def process_regions():
  """
    Obtiene y procesa datos básicos de todas las regiones
    Args:
        None
    Returns:
        regions_data: DataFrame con información de las regiones
  """
  print("Procesando regiones...")
  df = pd.read_csv('admin1CodesASCII.txt', sep='\t',
                    names=['code', 'name', 'name_ascii', 'geonameid'])

  df[['country_code', 'region_code']] = df['code'].str.split('.', expand=True)
  df['admin_type'] = 'REGION'

  return df[[
      'country_code', 'region_code', 'name',
      'admin_type', 'geonameid'
  ]]

In [ ]:
def enrich_with_continent(data_df, country_df):
  """
    Agrega información de continente a los datos
    Args:
        data_df: DataFrame con información de las regiones
        country_df: DataFrame con información de los países
    Returns:
        merged: DataFrame con información de las regiones y países
  """
  print("Agregando información de continente...")

  if 'continent_code' not in country_df.columns:
      print("Advertencia: 'continent_code' no encontrada en country_df, agregando columna predeterminada")
      country_df['continent_code'] = 'OC'

  if 'alpha_2_code' not in country_df.columns:
      print("Error: 'alpha_2_code' no encontrada en country_df")
      if 'cca2' in country_df.columns:
          country_df['alpha_2_code'] = country_df['cca2']
      else:
          return data_df

  if 'country_code' not in data_df.columns:
      print("Error: 'country_code' no encontrada en data_df")
      return data_df

  try:
      merged = data_df.merge(
          country_df[['alpha_2_code', 'continent_code']],
          left_on='country_code',
          right_on='alpha_2_code',
          how='left'
      )
      merged = merged.drop(columns=['alpha_2_code'])
      merged['continent_code'] = merged['continent_code'].fillna('OC')
      return merged
  except Exception as e:
      print(f"Error en la fusión: {e}")
      data_df['continent_code'] = 'OC'
  return data_df

In [ ]:
def split_and_save(data_df, base_name):
    """
    Divide y guarda los datos por continente
    Args:
        data_df: DataFrame con información de los datos
        base_name: Nombre base de los archivos
    Returns:
        None
    """
    print(f"Dividiendo y guardando {base_name}...")

    if 'continent_code' not in data_df.columns:
        print(f"Advertencia: 'continent_code' no encontrada en {base_name}_df, agregando columna predeterminada")
        data_df['continent_code'] = 'OC'

    data_df.to_csv(f"{BASE_PATH}all_{base_name}.csv", index=False, encoding='utf-8-sig')

    for continent_code in CONTINENT_MAP.keys():
        filtered = data_df[data_df['continent_code'] == continent_code]
        if not filtered.empty:
            filename = f"{BASE_PATH}{CONTINENT_MAP[continent_code].lower()}_{base_name}.csv"
            filtered.to_csv(filename, index=False, encoding='utf-8-sig')

In [ ]:
def main():
    # Verificar las dependencias
    try:
        import requests
        import pandas as pd
        import pycountry
        print("Todas las bibliotecas necesarias están disponibles")
    except ImportError as e:
        print(f"Error importando bibliotecas: {e}")
        return

    print("Iniciando ETL de datos geográficos...")

    # Paso 1: Procesar continentes (ahora se asigna un ID numérico)
    try:
        continents_df = get_continent_data()
        continents_df.to_csv(f"{BASE_PATH}continents.csv", index=False)
        print("Datos de continentes procesados")
    except Exception as e:
        print(f"Error procesando continentes: {e}")
        return

    # Paso 2: Procesar países
    try:
        countries_df = get_countries_data()
        if countries_df.empty:
            print("No se obtuvieron datos de países")
            return

        print(f"Se obtuvieron datos de {len(countries_df)} países")

        if 'continent_code' not in countries_df.columns:
            print("Error: 'continent_code' no está en countries_df")
            countries_df['continent_code'] = 'OC'

        countries_df = countries_df.merge(
            continents_df[['id', 'code']],
            left_on='continent_code',
            right_on='code',
            how='left'
        )
        countries_df.rename(columns={'id': 'continent_id'}, inplace=True)
        countries_df.drop(columns=['code'], inplace=True)

        try:
            split_and_save(countries_df, 'countries')
            print("Países guardados por continente")
        except Exception as e:
            print(f"Error guardando países: {e}")
    except Exception as e:
        print(f"Error general procesando países: {e}")
        return

    # Paso 3: Información adicional de países
    try:
        countries_info_df = get_countries_info_data(countries_df)
        if not countries_info_df.empty:
            split_and_save(countries_info_df, 'country_info')
            print("Información adicional de países procesada")
        else:
            print("No se obtuvo información adicional de países")
    except Exception as e:
        print(f"Error procesando información adicional: {e}")

    # Paso 4: Procesar regiones
    try:
        regions_df = process_regions()
        regions_df = enrich_with_continent(regions_df, countries_df)
        split_and_save(regions_df, 'regions')
        print("Regiones procesadas")
    except Exception as e:
        print(f"Error procesando regiones: {e}")

    # Paso 5: Procesar ciudades
    try:
        cities_df = process_geonames_cities()
        cities_df = enrich_with_continent(cities_df, countries_df)
        split_and_save(cities_df, 'cities')
        print("Ciudades procesadas")
    except Exception as e:
        print(f"Error procesando ciudades: {e}")

    # Paso 6: Ajustes finales en DataFrames para importación
    try:
        print("Realizando cambios finales...")
        df = pd.read_csv(f"{BASE_PATH}all_countries.csv", low_memory=False)
        df_countries_main = df[['alpha_2_code', 'alpha_3_code', 'numeric_code',
                                  'official_name', 'common_name', 'english_name',
                                  'continent_id', 'is_active', 'created_at', 'updated_at']]
        df_countries_main.to_csv(f"{BASE_PATH}countries.csv", index=False, encoding='utf-8')

        df_info = df[['alpha_2_code', 'capital', 'timezones', 'callingCode']].copy()
        df_info.rename(columns={'callingCode': 'calling_code', 'alpha_2_code': 'country_code'}, inplace=True)
        def fix_timezones(val):
            try:
                return json.loads(val) if isinstance(val, str) else val
            except Exception:
                return []
        df_info['timezones'] = df_info['timezones'].apply(fix_timezones)
        df_info.to_csv(f"{BASE_PATH}country_info.csv", index=False, encoding='utf-8')

        df_states = pd.read_csv(f"{BASE_PATH}all_regions.csv", low_memory=False)
        df_states_processed = df_states[['country_code', 'region_code', 'name', 'admin_type']].copy()
        df_states_processed.rename(columns={'region_code': 'code', 'name': 'official_name'}, inplace=True)
        df_states_processed.to_csv(f"{BASE_PATH}states.csv", index=False, encoding='utf-8')

        df_cities = pd.read_csv(f"{BASE_PATH}all_cities.csv", low_memory=False)
        df_cities_main = df_cities[['geonameid', 'name', 'country_code', 'admin1_code',
                                    'is_capital', 'is_state_capital']].copy()
        df_cities_main.rename(columns={'name': 'official_name'}, inplace=True)
        df_cities_main.to_csv(f"{BASE_PATH}cities_main.csv", index=False, encoding='utf-8')

        df_cities_info = df_cities[['geonameid', 'latitude', 'longitude', 'population', 'timezone']].copy()
        df_cities_info.to_csv(f"{BASE_PATH}cities_info_main.csv", index=False, encoding='utf-8')

        print("Cambios finales realizados")
    except Exception as e:
        print(f"Error realizando cambios finales: {e}")


    # Paso 7: Crear archivo comprimido
    try:
        print("Comprimiendo resultados...")
        !zip -r /content/geodata.zip {BASE_PATH}
        print("Archivo comprimido creado en /content/geodata.zip")
    except Exception as e:
        print(f"Error comprimiendo resultados: {e}")

    print("Proceso ETL completado!")

In [ ]:
if __name__ == "__main__":
    main()